In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/cse256_final_project_huayueli_chengxiguo/huayue_li")

In [ ]:
from dataset import ClothDataset

In [ ]:
cloth_dataset = ClothDataset("/content/drive/MyDrive/cse256_final_project_huayueli_chengxiguo/huayue_li/renttherunway_final_data.json.gz")
cloth_dataset.load_data()
cloth_dataset.preprocess_data()

Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[

118


In [ ]:
reviews = cloth_dataset.reviews[:5000]
ratings = cloth_dataset.ratings[:5000]


import gzip
import json
renttherunway_data = []
renttherunway_file = gzip.open("/content/drive/MyDrive/cse256_final_project_huayueli_chengxiguo/huayue_li/renttherunway_final_data.json.gz")
for rtrw_item in renttherunway_file:
    renttherunway_data.append(json.loads(rtrw_item))
for item in renttherunway_data:
  if int(item["rating"]) <= 4:
    print(item["rating"])
    print(item["review_summary"])

4
strange fit
4
Nice, but not for curves
2
Wasn't able to wear
4
Not cute on an hourglass shape
2
Too small-couldn't zip up. 
4
awkward shape for curvy bodies, even in the larger sizes. 
4
Pretty color.
4
Fun Jumpsuit!
2
nothing
4
Brought attention to my belly! No bueno
4
I wasn't able to wear and sent right back
4
Wasn't my favorite
4
Ill-fitting.
4
Adorbale, but not practical.
4
Cute dress, but too busty
4
Not for longer-torsos!
2
Ended up not wearing
4
Pockets!!
4
This was 1 of 3 dresses I ordered in my first unlimited shipment, rented to wear to work.
4
pretty dress but wouldn't fit
4
Really cute dress for the right body.
4
It was frumpy on my petite frame. 
4
Did not end up wearing this due to the large slit in the front of the dress!
4
Too too short!
4
got a lot of compliments
4
Did not fit.
4
Not a fan
4
I didn't end up wearing this dress
4
Great venue, great company!
4
Too long, and too wide
4
Really Uncomfortable
4
Very Pretty, but very bold.
4
Too small
2
Super fun, well made

TypeError: ignored

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

In [ ]:
max_length = max(len(review) for review in reviews)

zero_list = []
for i in range(100):
  zero_list.append(0)
print(max_length)

59


In [ ]:
for i in range(len(reviews)):
  current_length = len(reviews[i])
  padd_length = max_length - current_length
  for j in range(padd_length):
    reviews[i].append(zero_list)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class ReviewDataset(Dataset):
    def __init__(self, reviews, ratings):
        self.reviews = reviews
        self.ratings = ratings

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        return self.reviews[idx], self.ratings[idx]

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        y = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out1, out2 = self.rnn(x, y)
        out = self.fc(out1[:, -1, :])
        return out

In [ ]:
reviews_tensor = torch.tensor(reviews, dtype=torch.float32)
ratings_tensor = torch.tensor(ratings, dtype=torch.float32)

In [ ]:
dataset = ReviewDataset(reviews_tensor, ratings_tensor)

In [ ]:
input_size = 100
hidden_size = 64
num_layers = 2
output_size = 1
batch_size = 1
num_epochs = 100

In [ ]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = RNN(input_size, hidden_size, num_layers, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Epoch [1/100], Loss: 1.4016
Epoch [2/100], Loss: 1.5049
Epoch [3/100], Loss: 1.3274
Epoch [4/100], Loss: 0.4854
Epoch [5/100], Loss: 0.6230
Epoch [6/100], Loss: 1.1053
Epoch [7/100], Loss: 0.8205
Epoch [8/100], Loss: 0.9887
Epoch [9/100], Loss: 1.3842
Epoch [10/100], Loss: 0.8481
Epoch [11/100], Loss: 0.9590
Epoch [12/100], Loss: 0.8304
Epoch [13/100], Loss: 1.3108
Epoch [14/100], Loss: 0.9718
Epoch [15/100], Loss: 0.6566
Epoch [16/100], Loss: 0.8603
Epoch [17/100], Loss: 0.8531
Epoch [18/100], Loss: 0.3477
Epoch [19/100], Loss: 1.0626
Epoch [20/100], Loss: 0.9116
Epoch [21/100], Loss: 0.4348
Epoch [22/100], Loss: 0.7414
Epoch [23/100], Loss: 0.2565
Epoch [24/100], Loss: 0.9839
Epoch [25/100], Loss: 0.8297
Epoch [26/100], Loss: 0.4677
Epoch [27/100], Loss: 1.2237
Epoch [28/100], Loss: 1.1907
Epoch [29/100], Loss: 0.6994
Epoch [30/100], Loss: 0.7946
Epoch [31/100], Loss: 1.6231
Epoch [32/100], Loss: 1.1368
Epoch [33/100], Loss: 0.8713
Epoch [34/100], Loss: 1.1027
Epoch [35/100], Loss: 0

In [ ]:
torch.save(model.state_dict(),"/content/drive/MyDrive/cse256_final_project_huayueli_chengxiguo/huayue_li/model.pkl")